In [9]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Embedding, ELU, Add, Softmax, LayerNormalization, LayerNormalization, MultiHeadAttention
from tensorflow.keras.models import Model
import numpy as np
import tensorflow.keras as keras
import importlib
from tqdm import tqdm
import os

In [108]:
import models
import dataprocessor
importlib.reload(models)
importlib.reload(dataprocessor)
from models import *
from dataprocessor import *

macd_timescales = [(8, 24), (16, 28), (32, 96)]
rtn_timescales = [1, 21, 63, 126, 252]
timesteps = 16

folder_path = 'data'
files = [f for f in os.listdir(folder_path) if f.endswith('.xlsx')]
files = [file for file in files if file not in ("CC00.NYB.xlsx", "LB00.CME.xlsx", "ES00.CME.xlsx", "NQ00.CME.xlsx", "YM00.CBT.xlsx")]
data_list = []


data_list = process_data_list(files, macd_timescales, rtn_timescales, test = False)

处理文件中。。: 100%|██████████| 12/12 [00:50<00:00,  4.21s/it]


In [109]:
import models
import dataprocessor
importlib.reload(models)
importlib.reload(dataprocessor)
from models import *
from dataprocessor import *

In [110]:
target_set, labels = generate_tensors(data_list, timesteps, return_labels=True)
context_set = generate_tensors(data_list, timesteps, contain_next_day_rtn=True)
mean_set, std_set = labels

target_features, target_dates, target_info = target_set
context_features, context_dates, side_info = context_set
target_info, index_word_map = side_info_tensor_encoder(target_info)
side_info, _ = side_info_tensor_encoder(side_info)
target_set = (target_features, target_dates, target_info)
context_set = (context_features, context_dates, side_info)

In [ ]:
def date_to_int(date):
    date_int = tf.strings.to_number(tf.strings.regex_replace(date, "-", ""), tf.int32)
    return date_int


def find_matching_context(target_date, target_info, context_dates, context_features, side_info):
    mask_date = tf.math.greater(date_to_int(target_date), date_to_int(context_dates))
    mask_info = tf.reduce_all(tf.math.equal(target_info, side_info), axis=1)
    mask_info = tf.logical_not(mask_info)
    mask = tf.logical_and(mask_date, mask_info)
    valid_indices = tf.where(mask)
    if tf.size(valid_indices) > 0:
        random_index = tf.random.uniform(shape=[], minval=0, maxval=tf.size(valid_indices), dtype=tf.int32)
        selected_index = valid_indices[random_index, 0]
        return context_features[selected_index], context_dates[selected_index], side_info[selected_index]
    else:
        return None, None, None

matched_context_features = []
matched_context_dates = []
matched_side_info = []
matched_target_features = []
matched_target_dates = []
matched_target_info = []
target_distribution = []

for i in tqdm(range(len(target_dates))):
    target_date = target_dates[i]
    target_inf = target_info[i]
    target_feature = target_features[i]
    mean, std = mean_set[i], std_set[i]
    
    context_feature, context_date, side_inf = find_matching_context(
        target_date, target_inf, context_dates, context_features, side_info)
    
    if context_feature is not None:
        matched_context_features.append(context_feature)
        matched_context_dates.append(context_date)
        matched_side_info.append(side_inf)
        matched_target_features.append(target_feature)
        matched_target_dates.append(target_date)
        matched_target_info.append(target_inf)
        target_distribution.append((mean, std))

if matched_context_features:
    matched_context_features = tf.stack(matched_context_features)
    matched_context_dates = tf.stack(matched_context_dates)
    matched_side_info = tf.stack(matched_side_info)
    matched_target_features = tf.stack(matched_target_features)
    matched_target_dates = tf.stack(matched_context_dates)
    matched_target_info = tf.stack(matched_side_info)
    target_distribution = tf.stack(target_distribution)
    
else:
    matched_context_features = tf.constant([])
    matched_context_dates = tf.constant([])
    matched_side_info = tf.constant([])


In [ ]:
def expand_info(matched_side_info, time_steps):
    expanded_info = tf.expand_dims(matched_side_info, axis=1)  # Shape will be [batch, 1, feature]
    tiled_info = tf.tile(expanded_info, [1, time_steps, 1])
    return tiled_info

In [ ]:
import models
import dataprocessor
importlib.reload(models)
importlib.reload(dataprocessor)
from models import *
from dataprocessor import *

In [ ]:
import models
import dataprocessor
importlib.reload(models)
importlib.reload(dataprocessor)
from models import *
from dataprocessor import *

sequence_length = 8
hidden_dim = 128
encoding_size = 13

model = ModelWrapper(sequence_length, hidden_dim, encoding_size, 4)
model.build((matched_target_features.shape, expand_info(matched_target_info, 16).shape))
model(matched_context_features[:, :, 1:], matched_context_features, expand_info(matched_side_info, 16), matched_target_features, expand_info(matched_target_info, 16))